In [1]:
from ovcg import *
from fesol import *
import numpy as np
import scipy.io as sio
from interface import DynamicUnderRelaxation, InterfaceData, RelativeCovergenceMonitor

In [2]:
k=20
# problem discription:
# basically, with a fixed Pr=0.01 and Re=500, the k is the ratio between
# kappa_s/kappa_f, where kappa stands for conductivity coefficient.
assert k == 1 or k == 5 or k == 20

# tolerance setting
tol = 1e-3
if tol > 1e-1:
    print('The tolerance is too large, this may lead to instability or low quality results')

In [3]:
# since this is a matching interface, we can create
# exact interface nodes mappings.
# this step is done offline
inodes = 100
s2f = np.zeros(inodes, dtype=int)
f2s = s2f.copy()
s2f[1] = inodes-1
start = inodes-2
for i in range(inodes-2):
    s2f[i+2] = start
    start = start - 1
count = 0
for i in range(inodes):
    f2s[s2f[i]] = i

In [4]:
# fluid solver
interface_grid = 3
interface_side = 0
interface_axis = 1

solverF = Cgins()
solverF.read_cmd('fluid_inputs/fluid_flat_plate%i.cmd' % k)
solverF.init_grid()
solverF.init_temperature_interface(
    grid=interface_grid,
    side=interface_side,
    axis=interface_axis
)
solverF.init_solver(debug=0, logname='fluid_outputs/dummy')
fluid_ofile = VtkFile('fluid_outputs/resFPSPJJ%i.pvd' % k)

In [5]:
# solid solver
fixed_temp_tag = 2
adiabatic_tag = 3
interface_tag = 4

kappa = 100.0
rho = 1.0
Cp = 100.0
T_bot = 310.0

solverS = HeatSolver(kappa=kappa, rho=rho, Cp=Cp)
solverS.output_file = 'solid_outputs/resFPSPJJ%i.pvd' % k
solverS.load_mesh('solid_inputs/solid_plate.msh')
solverS.set_initial_condition(T_bot)
solverS.define_adiabatic_bd(tag=adiabatic_tag)
solverS.define_const_temperature_bd(tag=fixed_temp_tag, value=T_bot)
solverS.define_temperature_interface(tag=interface_tag)
solverS.init_solver()

In [6]:
# coupling data and setups
abTempF = InterfaceData(size=inodes, value=300.0)
abTempS = InterfaceData(size=inodes, value=T_bot)
TempF = InterfaceData(size=inodes, value=300.0)
TempS = InterfaceData(size=inodes, value=T_bot)

under_relax = DynamicUnderRelaxation(init_omega=1.0)
conv_mntr = RelativeCovergenceMonitor(tol=tol)

# maximum pc steps allowed
max_pc_steps = 200

# since this problem has no mesh motion and both kappa are material properties
# so the heat transfer coefficients don't vary in time
hF = solverF.get_heat_transfer_coeff(
    grid=interface_grid,
    side=interface_side,
    axis=interface_axis
)
hF2S = hF[f2s]
hS = solverS.get_interface_h(tag=interface_tag)
hS2F = hS[s2f]

CF1 = hS2F/(hS2F+hF)
CF2 = hF/(hS2F+hF)

CS1 = hS/(hF2S+hS)
CS2 = hF2S/(hF2S+hS)

In [7]:
# coupling timing information, use fixed time step for now
Tfinal = 1.0
N = 100
dt = Tfinal/N
t = 0.0
avg_pc_iterations = 0
flog = open('FPSPJJ%i.log' % k, mode='w')
flog.write('Fluid pseudo Robin with solid pseudo Robin setting, study case: %i\n' % k)
flog.close()
flog = open('FPSPJJ%i.log' % k, mode='a')

In [8]:
# begin to couple
for step in range(N):
    t += dt
    
    pc_counts = 0
    
    # back up solutions
    solverF.backup_current_solutions()
    solverS.backup()
    
    while True:
        pc_counts = pc_counts + 1
        
        # back up previous interface value
        TempF.backup()
        TempS.backup()
        
        # advance solvers
        solverF.advance(t=t, dt=dt)
        solverS.advance(t=t, dt=dt)
        
        # retrieve fluid ambient temperature
        abTempF.curr[:] = solverF.get_ambient_temperature(
            grid=interface_grid,
            side=interface_side,
            axis=interface_axis
        )
        
        # retrieve solid ambient temperature
        abTempS.curr[:] = solverS.get_interface_ambient_temperature(
            tag=interface_tag,
            h_given = hS
        )
        
        # update solid interface temperature
        TempS.curr[:] = CS1 * abTempS.curr + CS2 * abTempF.curr[f2s]
        # update fluid interface temperature
        TempF.curr[:] = CF1 * abTempS.curr[s2f] + CF2 * abTempF.curr
        
        # update residual
        TempF.update_res()
        TempS.update_res()
        
        # test convergence
        is_conv = conv_mntr.determine_convergence(TempF)
        if is_conv or pc_counts >= max_pc_steps:
            solverF.put_temperature_interface(
                grid=interface_grid,
                side=interface_side,
                axis=interface_axis,
                data=TempF.curr
            )
            solverS.update_temperature_interface(
                tag=interface_tag,
                T=TempS.curr
            )
            break
        else:
            # if not converge, then underrelaxation and update to fluid then restore
            under_relax.determine_omega(TempF)
            under_relax.update_solution(TempF)
            under_relax.determine_omega(TempS)
            under_relax.update_solution(TempS)
            solverF.put_temperature_interface(
                grid=interface_grid,
                side=interface_side,
                axis=interface_axis,
                data=TempF.curr
            )
            solverS.update_temperature_interface(
                tag=interface_tag,
                T=TempS.curr
            )
            solverF.restore_previous_solutions()
            solverS.restore()  

    # outputs
    msg = 'step=%i, pc_iterations=%i.' % (step, pc_counts)
    print(msg)
    flog.write(msg+'\n')
    solverS.write()
    fluid_ofile.write(solverF, t)
    avg_pc_iterations = avg_pc_iterations + pc_counts
flog.close()

step=0, pc_iterations=4.
step=1, pc_iterations=1.
step=2, pc_iterations=1.
step=3, pc_iterations=1.
step=4, pc_iterations=1.
step=5, pc_iterations=1.
step=6, pc_iterations=1.
step=7, pc_iterations=1.
step=8, pc_iterations=1.
step=9, pc_iterations=1.
step=10, pc_iterations=1.
step=11, pc_iterations=1.
step=12, pc_iterations=1.
step=13, pc_iterations=1.
step=14, pc_iterations=1.
step=15, pc_iterations=1.
step=16, pc_iterations=1.
step=17, pc_iterations=1.
step=18, pc_iterations=1.
step=19, pc_iterations=1.
step=20, pc_iterations=1.
step=21, pc_iterations=1.
step=22, pc_iterations=1.
step=23, pc_iterations=1.
step=24, pc_iterations=1.
step=25, pc_iterations=1.
step=26, pc_iterations=1.
step=27, pc_iterations=1.
step=28, pc_iterations=1.
step=29, pc_iterations=1.
step=30, pc_iterations=1.
step=31, pc_iterations=1.
step=32, pc_iterations=1.
step=33, pc_iterations=1.
step=34, pc_iterations=1.
step=35, pc_iterations=1.
step=36, pc_iterations=1.
step=37, pc_iterations=1.
step=38, pc_iterations

In [9]:
theta=(solverF.get_temperature_interface(3,0,1)-300)/10
print(theta)
sio.savemat('numeric_res/kFPSPJJ%i.mat' % k, {'theta': theta, 'avg_pc_its': avg_pc_iterations/N})

[0.88722062 0.89332628 0.89872176 0.9017895  0.90439505 0.90676984
 0.90884347 0.91069234 0.91245824 0.91399771 0.91541538 0.91671024
 0.91794591 0.91907286 0.92012531 0.92109055 0.92200949 0.92286161
 0.92364645 0.92439205 0.92508013 0.92572547 0.92633078 0.92688802
 0.92741496 0.92790049 0.92836478 0.92878571 0.92917267 0.92953888
 0.92988213 0.93019627 0.9304908  0.930761   0.93101403 0.93124382
 0.93145895 0.93165368 0.93183987 0.93200627 0.9321578  0.93229334
 0.93241761 0.93253123 0.93263352 0.93272434 0.93280569 0.93287608
 0.93293486 0.93298774 0.93302947 0.93306384 0.93308847 0.93310506
 0.93311345 0.93311537 0.93310758 0.93309155 0.93306818 0.93303576
 0.93299287 0.93294471 0.93288554 0.93281877 0.93274019 0.93265378
 0.93255509 0.93244702 0.93232482 0.93219211 0.9320499  0.93189135
 0.93171277 0.93152562 0.93131803 0.93109564 0.93084936 0.93058522
 0.93029543 0.92998436 0.92964241 0.92927234 0.928873   0.92844183
 0.92796897 0.92744503 0.92689289 0.92627516 0.92558914 0.9248